<p> <center> <a href="../start_here.ipynb.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="02_introduction_mcp.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="01_inference_endpoint.ipynb">1</a>
        <a href="02_introduction_mcp.ipynb">2</a>
        <a >3</a>
        <a href="04_langraph_agent.ipynb">4</a>
        <a href="05_challenge.ipynb">5</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="04_langraph_agent.ipynb">Next Notebook</a></span>
</div>

## Learning Objectives

By the end of this notebook, participants will be able to:

* Understand the difference between high-level (FastMCP) and low-level MCP SDK
* Build MCP servers using the low-level SDK with explicit tool definitions
* Define tool schemas manually using `types.Tool` with JSON Schema
* Implement custom tool handlers with input validation and error handling
* Deploy low-level MCP servers using both Stdio and HTTP transports
* Connect to low-level servers using the MCP client

## Low-Level MCP Server Implementation
In the previous notebook, we used **FastMCP** (high-level SDK) to quickly build MCP servers. Now, let's explore the **low-level SDK**, which offers full control over server behavior at the cost of more verbose code.

Here's when to use each approach:

| Feature | High-Level (FastMCP) | Low-Level SDK |
|---------|---------------------|---------------|
| **Tool Definition** | `@mcp.tool()` decorator | Manual `types.Tool` schema |
| **Input Validation** | Automatic from type hints | Manual validation required |
| **Boilerplate** | Minimal | More verbose |
| **Control** | Abstracted | Full control |
| **Use Case** | Quick prototyping | Production, custom behavior |

### Low-Level MCP Server with Stdio Transport

Let's build an MCP server using the low-level SDK. Unlike FastMCP where we simply use decorators, here we need to:

1. **Create a `Server` instance** – The core server object
2. **Define `@server.list_tools()`** – Manually specify tool schemas using JSON Schema
3. **Define `@server.call_tool()`** – Handle tool execution with manual validation
4. **Set up the transport** – Initialize stdio streams for communication

In [ ]:
%%writefile mcp_server_low_level.py
"""
Low-Level MCP Server Implementation (Stdio Transport)

This server demonstrates how to build MCP servers with full control
over tool definitions, input validation, and error handling.
"""

import asyncio
import logging
from typing import Any

from mcp.server import InitializationOptions
from mcp.server.lowlevel import Server, NotificationOptions
from mcp.server.stdio import stdio_server
import mcp.types as types

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('simple-math')


async def main():
    # ===========================================
    # Step 1: Create the Server Instance
    # ===========================================
    server = Server("simple-math")

    # ===========================================
    # Step 2: Define Available Tools
    # ===========================================
    @server.list_tools()
    async def handle_list_tools() -> list[types.Tool]:
        """
        Returns the name, description, and input schema of all available tools.
        
        Each tool requires:
        - name: Unique identifier for the tool
        - description: Human-readable description
        - inputSchema: JSON Schema defining expected arguments
        """
        return [
            types.Tool(
                name="add",
                description="Add two numbers together",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First number"},
                        "b": {"type": "number", "description": "Second number"}
                    },
                    "required": ["a", "b"],
                },
            ),
            types.Tool(
                name="subtract",
                description="Subtract second number from first",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First number"},
                        "b": {"type": "number", "description": "Second number"}
                    },
                    "required": ["a", "b"]
                },
            ),
        ]

    # ===========================================
    # Step 3: Implement Tool Handlers
    # ===========================================
    @server.call_tool()
    async def handle_call_tool(
        name: str, arguments: dict[str, Any] | None
    ) -> list[types.TextContent | types.ImageContent | types.EmbeddedResource]:
        """
        Handle tool execution requests.
        
        Args:
            name: The name of the tool to execute
            arguments: Dictionary of arguments passed to the tool
            
        Returns:
            List of content objects (TextContent, ImageContent, etc.)
        """
        try:
            # Validate arguments exist
            if not arguments:
                raise ValueError("No arguments provided")
            
            # Handle 'add' tool
            if name == "add":
                if "a" not in arguments or "b" not in arguments:
                    raise ValueError("Missing required arguments: a, b")
                result = arguments['a'] + arguments['b']
                return [types.TextContent(type="text", text=str(result))]

            # Handle 'subtract' tool
            elif name == "subtract":
                if "a" not in arguments or "b" not in arguments:
                    raise ValueError("Missing required arguments: a, b")
                result = arguments['a'] - arguments['b']
                return [types.TextContent(type="text", text=str(result))]
            
            else:
                raise ValueError(f"Unknown tool: {name}")
                
        except Exception as e:
            logger.error(f"Error executing tool '{name}': {e}")
            return [types.TextContent(type="text", text=f"Error: {str(e)}")]

    # ===========================================
    # Step 4: Initialize and Run the Server
    # ===========================================
    async with stdio_server() as (read_stream, write_stream):
        logger.info("Server running with stdio transport")
        await server.run(
            read_stream,
            write_stream,
            InitializationOptions(
                server_name="simple-math",
                server_version="0.1.0",
                capabilities=server.get_capabilities(
                    notification_options=NotificationOptions(),
                    experimental_capabilities={},
                ),
            ),
        )


# Entry point
if __name__ == "__main__":
    asyncio.run(main())

Let's use our MCP client to connect to the low-level server and invoke tools.

In [ ]:
import subprocess
import time

# Start the HTTP server in the background
server_process = subprocess.Popen(
    ["python", "mcp_server_low_level.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(3)

# Store and display the process ID
SERVER_PID = server_process.pid
print(f"Process ID: {SERVER_PID}")

Let's run the client to connect and invoke the tools.

In [ ]:
!python mcp_client.py mcp_server_low_level.py

Always clean up background processes when done.

In [ ]:
!kill -9 {SERVER_PID}
print(f"✓ Server (PID: {SERVER_PID}) killed")

### Low-Level MCP Server with HTTP Transport

For production deployments or remote access, we can use **HTTP transport** instead of Stdio. The server will be accessible at `http://localhost:8000/mcp`.

In [ ]:
%%writefile mcp_server_low_level_http.py
"""
Low-Level MCP Server Implementation (HTTP Transport)

Uses Starlette + Uvicorn for production-ready HTTP deployment.
Endpoint: http://localhost:8000/mcp
"""

import asyncio
import logging
import contextlib
from collections.abc import AsyncIterator
from typing import Any

from mcp.server import InitializationOptions
from mcp.server.lowlevel import Server, NotificationOptions
from mcp.server.streamable_http_manager import StreamableHTTPSessionManager
from starlette.applications import Starlette
from starlette.routing import Mount
from starlette.types import Receive, Scope, Send
import uvicorn
import mcp.types as types

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('simple-math')


def main():
    # ===========================================
    # Step 1: Create the Server Instance
    # ===========================================
    server = Server("simple-math")

    # ===========================================
    # Step 2: Define Available Tools
    # ===========================================
    @server.list_tools()
    async def handle_list_tools() -> list[types.Tool]:
        """List available tools with their schemas."""
        return [
            types.Tool(
                name="add",
                description="Add two numbers together",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First number"},
                        "b": {"type": "number", "description": "Second number"}
                    },
                    "required": ["a", "b"],
                },
            ),
            types.Tool(
                name="subtract",
                description="Subtract second number from first",
                inputSchema={
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First number"},
                        "b": {"type": "number", "description": "Second number"}
                    },
                    "required": ["a", "b"]
                },
            ),
        ]

    # ===========================================
    # Step 3: Implement Tool Handlers
    # ===========================================
    @server.call_tool()
    async def handle_call_tool(
        name: str, arguments: dict[str, Any] | None
    ) -> list[types.TextContent | types.ImageContent | types.EmbeddedResource]:
        """Handle tool execution requests."""
        try:
            if not arguments:
                raise ValueError("No arguments provided")
            
            if name == "add":
                if "a" not in arguments or "b" not in arguments:
                    raise ValueError("Missing required arguments: a, b")
                result = arguments['a'] + arguments['b']
                return [types.TextContent(type="text", text=str(result))]

            elif name == "subtract":
                if "a" not in arguments or "b" not in arguments:
                    raise ValueError("Missing required arguments: a, b")
                result = arguments['a'] - arguments['b']
                return [types.TextContent(type="text", text=str(result))]
            
            else:
                raise ValueError(f"Unknown tool: {name}")
                
        except Exception as e:
            logger.error(f"Error executing tool '{name}': {e}")
            return [types.TextContent(type="text", text=f"Error: {str(e)}")]

    # ===========================================
    # Step 4: Configure HTTP Session Manager
    # ===========================================
    session_manager = StreamableHTTPSessionManager(
        app=server,
        event_store=None,
        json_response=True,
        stateless=True,
    )

    # ===========================================
    # Step 5: Define Request Handler
    # ===========================================
    async def handle_streamable_http(
        scope: Scope, receive: Receive, send: Send
    ) -> None:
        """Handle incoming HTTP requests."""
        await session_manager.handle_request(scope, receive, send)

    # ===========================================
    # Step 6: Configure Application Lifecycle
    # ===========================================
    @contextlib.asynccontextmanager
    async def lifespan(app: Starlette) -> AsyncIterator[None]:
        """Manage application startup and shutdown."""
        async with session_manager.run():
            logger.info("✓ MCP Server started at http://127.0.0.1:8000/mcp")
            try:
                yield
            finally:
                logger.info("✓ MCP Server shutting down...")

    # ===========================================
    # Step 7: Create and Run ASGI Application
    # ===========================================
    starlette_app = Starlette(
        debug=True,
        routes=[
            Mount("/mcp", app=handle_streamable_http),
        ],
        lifespan=lifespan,
    )

    # Start the server
    uvicorn.run(starlette_app, host="0.0.0.0", port=8000)


# Entry point
if __name__ == "__main__":
    main()

In [ ]:
import subprocess
import time

# Start the HTTP server in the background
server_process = subprocess.Popen(
    ["python", "mcp_server_low_level_http.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(3)

# Store and display the process ID
SERVER_PID = server_process.pid
print(f"Process ID: {SERVER_PID}")

In [ ]:
!python mcp_client.py --http

In [ ]:
!kill -9 {SERVER_PID}
print(f"✓ Server (PID: {SERVER_PID}) killed")

### Links and Resources

In [ ]:
- [] 

---
## Licensing

Copyright © 2025 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.

<p> <center> <a href="../start_here.ipynb.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="02_introduction_mcp.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="01_inference_endpoint.ipynb">1</a>
        <a href="02_introduction_mcp.ipynb">2</a>
        <a >3</a>
        <a href="04_langraph_agent.ipynb">4</a>
        <a href="05_challenge.ipynb">5</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="04_langraph_agent.ipynb">Next Notebook</a></span>
</div>